# Step-by-step Guidance for CIKM 2022 AnalytiCup Competition

## Step1. Install FederatedScope

In [ ]:
# Copy FederatedScope and switch to the stable branch cikm22competitionfor competition

!cp -r fs_latest FederatedScope

# `cd FederatedScope` if you are NOT in the playground.
import os
os.chdir('FederatedScope')

!git checkout cikm22competition

## Step 2. Setup the running environment

In [ ]:
# Install FS as follows
!pip install -e . --user

## Step 3. Download contest data

In [ ]:
# We have downloaded the data in advance in `data`, unzip the contest data as follows:
!mkdir -p data
!cp -r ../data/CIKM22Competition ./data/

Then you can access the contest data with the directory FederatedScope/data/CIKM22Competition. The contest data is organized by the index of the client CIKM22Competition/${client_id}(counts from 1), and the data of each client contains the train, test and validate splitted parts. You can load it by torch.loadas follows

In [ ]:
import torch
# The train split of client 1
train_data_client1 = torch.load('./data/CIKM22Competition/1/train.pt')
# Check the first sample
print(train_data_client1[0])
# Check the label of the first sample
print(train_data_client1[0].y)
# Check the index of the first sample as ${sample_id}
print(train_data_client1[0].data_index)

## Step 4. Execute baselines on the contest data

Within FederatedScope, we build in two baselines for the contest data, "isolated training" and "FedAvg". Suppose you have successfully built the running environment, and downloaded the contest data

In [ ]:
# We only run 3 round in playground, you can modify it by yourself.
# Run the following command to execute the isolated training
!python federatedscope/main.py --cfg federatedscope/gfl/baseline/isolated_gin_minibatch_on_cikmcup.yaml --client_cfg federatedscope/gfl/baseline/isolated_gin_minibatch_on_cikmcup_per_client.yaml federate.total_round_num 3

# Run the following command to execute the FedAvg solution
!python federatedscope/main.py --cfg federatedscope/gfl/baseline/fedavg_gin_minibatch_on_cikmcup.yaml --client_cfg federatedscope/gfl/baseline/fedavg_gin_minibatch_on_cikmcup_per_client.yaml federate.total_round_num 3

where the argument--cfg xxxx.yaml specifies the global configuration, and --client_cfg xxx.yaml specifies the client-wise hyper parameters.

## Step 5. Save and submit the prediction results

### Submission format
As stated in the introduction of CIKM 2022 AnalytiCup Competition, participants are required to submit the prediction results for all clients within one csv file. Within the file, each line records one prediction and is identified by `${client_id}` and `${sample_id}`. The `${client_id}` counts from 1 and `${sample_id}` should be consistent with the contest data (You can access it by the attribute data_index).

The classification and multi-dimensional regression tasks follow different formats as follows
* For classification tasks, each line follows (`${category_id}` counts from 0)

```
${client_id},${sample_id},${category_id}
```

For N-dimensional regression task, each line follows
```
${client_id},${sample_id},${prediction_1st_dimension},…,${prediction_N-th_dimension}
```

### Saving prediction results
By FederatedScope
The "cikm22competition" branch in FederatedScope supports to save prediction results at the end of training. You can refer to code in federatedscope/gfl/trainer/graphtrainer.py and federatedscope/core/trainers/torch_trainer.py.
The prediction results will be saved in a csv file named prediction.csv. For the convenience of users to conduct multiple experiments (e.g., for HPO), prediction.csv of each experimental run with a spcific configuration will be placed in this experiment's output directory (specified by outdir), which will be automatically appended with a suffix of timestamp if the specified directory has been there.
The training log will report the directory of the prediction results. Taking FedAvg as an example, at the end of training FederatedScope will report the path of prediction results as follows.
![Finish saving prediction results](https://img.alicdn.com/imgextra/i1/O1CN01L68gve1nGu3BNp5vL_!!6000000005063-2-tps-4766-404.png)

### Submit prediction results
Finally, you can submit your prediction results and get your score in [Tianchi](https://tianchi.aliyun.com/competition/entrance/532008/introduction).